# what happens if I split the years and run the algos separately?

In [1]:
library(neuralnet)
#library(rms)
library(devtools)
library(stargazer)

Warning message:
"package 'neuralnet' was built under R version 3.4.4"Warning message:
"package 'devtools' was built under R version 3.4.3"
Please cite as: 

 Hlavac, Marek (2015). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2. http://CRAN.R-project.org/package=stargazer 



In [2]:
library(RCurl)
library(jsonlite)
library(caret)
library(ggplot2)
library(e1071)
library(h2o)
library(statmod)
library(MASS)
library(corrplot)
library(data.table)
library(ROSE)

Loading required package: bitops
Warning message:
"package 'caret' was built under R version 3.4.4"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.4.4"Warning message:
"package 'e1071' was built under R version 3.4.3"Warning message:
"package 'h2o' was built under R version 3.4.4"
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: 'h2o'

The following objects are masked from 'package:stats':

    cor, sd, var

The following objects are masked from 'package:base':

    %*%, %in%, &&, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor

# data preparation

In [4]:
# loading the data
ess1 <- read.table("C:/Users/laris/Desktop/MMDS/Semester3-FSS2018/Advanced-Quantitative-Methods/paper/ess1_with-vars-nolabel.csv", header=TRUE, sep=",") 

In [5]:
ess6 <- read.table("C:/Users/laris/Desktop/MMDS/Semester3-FSS2018/Advanced-Quantitative-Methods/paper/ess6_with-vars-nolabel.csv", header=TRUE, sep=",") 

In [6]:
for(i in c(4,5,6,10,11,12,13,14,15,16,17,18,19,20,21,22,23)){
    ess1[is.na(ess1[,i]), i] <- mean(ess1[,i], na.rm = TRUE)
}

In [7]:
ess1[is.na(ess1[,3]), 3] <- 1
ess1[is.na(ess1[,7]), 7] <- 1
ess1[is.na(ess1[,8]), 8] <- 0
ess1[is.na(ess1[,9]), 9] <- 0

In [8]:
for(i in c(4,5,6,10,11,12,13,14,15,16,17,18,19,20,21,22,23)){
    ess6[is.na(ess6[,i]), i] <- mean(ess6[,i], na.rm = TRUE)
}

In [9]:
ess6[is.na(ess6[,3]), 3] <- 1
ess6[is.na(ess6[,7]), 7] <- 1
ess6[is.na(ess6[,8]), 8] <- 0
ess6[is.na(ess6[,9]), 9] <- 0

In [10]:
ess1.scaled <- cbind(ess1[c(1, 3, 7, 8, 9)], scale(ess1[-c(1, 2, 3, 7, 8, 9, 24)]))

In [11]:
ess1c.scaled <- cbind(ess1[c(1, 2, 3, 7, 8, 9)], scale(ess1[-c(1, 2, 3, 7, 8, 9, 24)]))

In [12]:
ess6.scaled <- cbind(ess6[c(1, 3, 7, 8, 9)], scale(ess6[-c(1, 2, 3, 7, 8, 9, 24)]))

In [13]:
ess6c.scaled <- cbind(ess6[c(1, 2, 3, 7, 8, 9)], scale(ess6[-c(1, 2, 3, 7, 8, 9, 24)]))

In [14]:
for(t in unique(ess1c.scaled$cntry)) {
    ess1c.scaled[paste("cntry",t,sep="")] <- ifelse(ess1c.scaled$cntry==t,1,0)
}

In [15]:
ess1c.scaled <- ess1c.scaled[,-2]

In [16]:
for(t in unique(ess6c.scaled$cntry)) {
    ess6c.scaled[paste("cntry",t,sep="")] <- ifelse(ess6c.scaled$cntry==t,1,0)
}

In [17]:
ess6c.scaled <- ess6c.scaled[,-2]

In [18]:
write.csv(ess1.scaled, file = "ess1-scaled.csv")
write.csv(ess1c.scaled, file = "ess1-scaled-cntry.csv")

In [19]:
write.csv(ess6.scaled, file = "ess6-scaled.csv")
write.csv(ess6c.scaled, file = "ess6-scaled-cntry.csv")

In [3]:
PRE <- function(table){
    tp <- table[2,2]
    tn <- table[1,1]
    fp <- table[2,1]
    fn <- table[1,2]
    PCP <- (tp + tn)/(tp + tn + fp + fn)
    PMC <- max(tp + fn, tn + fp)/(tp + tn + fp + fn)
    result <- (PCP - PMC)/(1 - PMC)
    return(result)
}

In [4]:
F1 <- function(table){
    tp <- table[2,2]
    tn <- table[1,1]
    fp <- table[2,1]
    fn <- table[1,2]
    precision <- tp / (tp + fp)
    recall <- tp / (tp + fn)
    result <- (2 * precision * recall) / (precision + recall)
    return(result)
}

In [5]:
ACC <- function(table){
    tp <- table[2,2]
    tn <- table[1,1]
    fp <- table[2,1]
    fn <- table[1,2]
    acc <- (tn + tp) / (tn + tp + fn + fp)
    return(acc)
}

# data loading

In [6]:
ess1.scaled <- read.table("ess1-scaled.csv", header=TRUE, sep=",") 
ess6.scaled <- read.table("ess6-scaled.csv", header=TRUE, sep=",") 
ess1c.scaled <- read.table("ess1-scaled-cntry.csv", header=TRUE, sep=",") 
ess6c.scaled <- read.table("ess6-scaled-cntry.csv", header=TRUE, sep=",") 

In [7]:
ess1.scaled <- ess1.scaled[,-1]
ess6.scaled <- ess6.scaled[,-1]

In [8]:
ess1c.scaled <- ess1c.scaled[,-1]
ess6c.scaled <- ess6c.scaled[,-1]

In [9]:
set.seed(123)
smp_size <- floor(0.9 * nrow(ess1.scaled))
train_ind <- sample(seq_len(nrow(ess1.scaled)), size = smp_size)
train1 <- ess1.scaled[train_ind,]
test1 <- ess1.scaled[-train_ind,]
train1c <- ess1c.scaled[train_ind,]
test1c <- ess1c.scaled[-train_ind,]

In [10]:
set.seed(123)
smp_size <- floor(0.9 * nrow(ess6.scaled))
train_ind <- sample(seq_len(nrow(ess6.scaled)), size = smp_size)
train6 <- ess6.scaled[train_ind,]
test6 <- ess6.scaled[-train_ind,]
train6c <- ess6c.scaled[train_ind,]
test6c <- ess6c.scaled[-train_ind,]

In [28]:
#train.balanced <- ovun.sample(volact ~ ., data = train, method = "over", N = 114264)$data

In [29]:
#table(train.balanced$volact)

# logistic regression

In [30]:
ll_logit <- function(theta, y, X) {
    
  # theta consists merely of beta (dim is ncol(X))
    beta <- theta[1:ncol(X)]
    # linear predictor; make sure that X is stored as.matrix
    mu <- X %*% beta
    # response function
    p <- 1/(1 + exp(-mu))
    # log-likelihood
    ll <- y * log(p) + (1 - y) * log(1 - p)
    # sum
    ll <- sum(ll)
    return(ll)
}

In [31]:
y <- train1$volact
X <- as.matrix(cbind(1, train1[,-1]))
startvals <- c(rep(0, 22))
res <- optim(par = startvals,fn = ll_logit, y = y, X = X,
              control = list(fnscale = -1),
              method = "BFGS"
              ) 

In [32]:
X2 <- as.matrix(cbind(1, test1[,-1]))
mu <- X2 %*% res$par
p <- 1/(1 + exp(-mu))
y_hat1 <- rbinom(nrow(p), 1, p)

In [33]:
table(y_hat1, test1$volact)
ACC(table(y_hat1, test1$volact))
PRE(table(y_hat1, test1$volact))
F1(table(y_hat1, test1$volact))

      
y_hat1    0    1
     0 1978  860
     1  863  535

[1] 0.5932483

[1] -0.2351254

[1] 0.3831006

In [34]:
y <- train6$volact
X <- as.matrix(cbind(1, train6[,-1]))
startvals <- c(rep(0, 22))
res <- optim(par = startvals,fn = ll_logit, y = y, X = X,
              control = list(fnscale = -1),
              method = "BFGS"
              ) 

In [35]:
X2 <- as.matrix(cbind(1, test6[,-1]))
mu <- X2 %*% res$par
p <- 1/(1 + exp(-mu))
y_hat6 <- rbinom(nrow(p), 1, p)

In [36]:
table(y_hat6, test6$volact)
ACC(table(y_hat6, test6$volact))
PRE(table(y_hat6, test6$volact))
F1(table(y_hat6, test6$volact))

      
y_hat6    0    1
     0 2392 1128
     1 1078  870

[1] 0.5965618

[1] -0.1041041

[1] 0.4409529

# neural net

In [11]:
c1 <- h2o.init()


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    C:\Users\laris\AppData\Local\Temp\RtmpsxUg2n/h2o_laris_started_from_r.out
    C:\Users\laris\AppData\Local\Temp\RtmpsxUg2n/h2o_laris_started_from_r.err


Starting H2O JVM and connecting: ... Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         9 seconds 461 milliseconds 
    H2O cluster timezone:       Europe/Berlin 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.18.0.8 
    H2O cluster version age:    1 month and 11 days  
    H2O cluster name:           H2O_started_from_R_laris_pfs973 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   1.66 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FAL

In [12]:
train1$volact <- as.factor(train1$volact)
test1$volact <- as.factor(test1$volact)
train6$volact <- as.factor(train6$volact)
test6$volact <- as.factor(test6$volact)

h2o.train1 <- as.h2o(train1)
h2o.test1 <- as.h2o(test1)
h2o.train6 <- as.h2o(train6)
h2o.test6 <- as.h2o(test6)

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [13]:
train1c$volact <- as.factor(train1c$volact)
test1c$volact <- as.factor(test1c$volact)
train6c$volact <- as.factor(train6c$volact)
test6c$volact <- as.factor(test6c$volact)

h2o.train1c <- as.h2o(train1c)
h2o.test1c <- as.h2o(test1c)
h2o.train6c <- as.h2o(train6c)
h2o.test6c <- as.h2o(test6c)

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [14]:
xnames1 <- colnames(train1[,-1])
xnames6 <- colnames(train6[,-1])

In [15]:
xnames1c <- colnames(train1c[,-1])
xnames6c <- colnames(train6c[,-1])

In [42]:
nn1_1 <- h2o.deeplearning(
  x = xnames1,
  y = "volact",
  training_frame = h2o.train1,
  validation_frame = h2o.test1,
  activation = "Rectifier",
  hidden = c(200, 200, 100),
  epochs = 10,
  rate = .005,
  loss = "CrossEntropy",
  export_weights_and_biases = TRUE,
  seed = 1000000
)

nn6_1 <- h2o.deeplearning(
  x = xnames6,
  y = "volact",
  training_frame = h2o.train6,
  validation_frame = h2o.test6,
  activation = "Rectifier",
  hidden = c(200, 200, 100),
  epochs = 10,
  rate = .005,
  loss = "CrossEntropy",
  export_weights_and_biases = TRUE, 
  seed = 1000000
)

  |======================================================================| 100%
  |======================================================================| 100%


In [43]:
summary(nn1_1)

Model Details:

H2OBinomialModel: deeplearning
Model Key:  DeepLearning_model_R_1527753908592_473 
Status of Neuron Layers: predicting volact, 2-class classification, bernoulli distribution, CrossEntropy loss, 64.902 weights/biases, 773,7 KB, 381.230 training samples, mini-batch size 1
  layer units      type dropout       l1       l2 mean_rate rate_rms momentum
1     1    21     Input  0.00 %                                              
2     2   200 Rectifier  0.00 % 0.000000 0.000000  0.023449 0.007265 0.000000
3     3   200 Rectifier  0.00 % 0.000000 0.000000  0.011245 0.006026 0.000000
4     4   100 Rectifier  0.00 % 0.000000 0.000000  0.030819 0.054879 0.000000
5     5     2   Softmax         0.000000 0.000000  0.008541 0.004004 0.000000
  mean_weight weight_rms mean_bias bias_rms
1                                          
2    0.018726   0.155087  0.324421 0.096519
3   -0.025297   0.072099  0.925869 0.049365
4   -0.008851   0.084945 -0.074801 0.270474
5   -0.011092   0.373333 

In [44]:
summary(nn6_1)

Model Details:

H2OBinomialModel: deeplearning
Model Key:  DeepLearning_model_R_1527753908592_490 
Status of Neuron Layers: predicting volact, 2-class classification, bernoulli distribution, CrossEntropy loss, 64.902 weights/biases, 773,7 KB, 492.050 training samples, mini-batch size 1
  layer units      type dropout       l1       l2 mean_rate rate_rms momentum
1     1    21     Input  0.00 %                                              
2     2   200 Rectifier  0.00 % 0.000000 0.000000  0.025428 0.012053 0.000000
3     3   200 Rectifier  0.00 % 0.000000 0.000000  0.015898 0.007762 0.000000
4     4   100 Rectifier  0.00 % 0.000000 0.000000  0.036133 0.057225 0.000000
5     5     2   Softmax         0.000000 0.000000  0.009030 0.005050 0.000000
  mean_weight weight_rms mean_bias bias_rms
1                                          
2    0.009545   0.192716  0.275757 0.112267
3   -0.029518   0.074397  0.909455 0.085216
4   -0.006929   0.084919 -0.164714 0.254429
5   -0.014307   0.367444 

In [45]:
nn1_1_pred <- h2o.predict(nn1_1, h2o.test1)
nn6_1_pred <- h2o.predict(nn6_1, h2o.test6)
ACC(table(as.vector(nn1_1_pred$predict), test1$volact))
PRE(table(as.vector(nn1_1_pred$predict), test1$volact))
F1(table(as.vector(nn1_1_pred$predict), test1$volact))
ACC(table(as.vector(nn6_1_pred$predict), test6$volact))
PRE(table(as.vector(nn6_1_pred$predict), test6$volact))
F1(table(as.vector(nn6_1_pred$predict), test6$volact))

  |======================================================================| 100%
  |======================================================================| 100%


[1] 0.5833333

[1] -0.265233

[1] 0.5482467

[1] 0.6126554

[1] -0.06006006

[1] 0.593786

In [46]:
nn1_1c <- h2o.deeplearning(
  x = xnames1c,
  y = "volact",
  training_frame = h2o.train1c,
  validation_frame = h2o.test1c,
  activation = "Rectifier",
  hidden = c(200, 200, 100),
  epochs = 10,
  rate = .005,
  loss = "CrossEntropy",
  export_weights_and_biases = TRUE,
  seed = 1000000
)

nn6_1c <- h2o.deeplearning(
  x = xnames6c,
  y = "volact",
  training_frame = h2o.train6c,
  validation_frame = h2o.test6c,
  activation = "Rectifier",
  hidden = c(200, 200, 100),
  epochs = 10,
  rate = .005,
  loss = "CrossEntropy",
  export_weights_and_biases = TRUE, 
  seed = 1000000
)

  |======================================================================| 100%
  |======================================================================| 100%


In [47]:
summary(nn1_1c)

Model Details:

H2OBinomialModel: deeplearning
Model Key:  DeepLearning_model_R_1527753908592_505 
Status of Neuron Layers: predicting volact, 2-class classification, bernoulli distribution, CrossEntropy loss, 69.302 weights/biases, 828,8 KB, 381.230 training samples, mini-batch size 1
  layer units      type dropout       l1       l2 mean_rate rate_rms momentum
1     1    43     Input  0.00 %                                              
2     2   200 Rectifier  0.00 % 0.000000 0.000000  0.041933 0.039444 0.000000
3     3   200 Rectifier  0.00 % 0.000000 0.000000  0.011634 0.006996 0.000000
4     4   100 Rectifier  0.00 % 0.000000 0.000000  0.037490 0.098025 0.000000
5     5     2   Softmax         0.000000 0.000000  0.006113 0.004200 0.000000
  mean_weight weight_rms mean_bias bias_rms
1                                          
2   -0.020447   0.175266  0.399573 0.094940
3   -0.021160   0.073089  0.946080 0.048659
4   -0.011110   0.086024  0.779643 0.080492
5   -0.013758   0.440890 

In [48]:
summary(nn6_1c)

Model Details:

H2OBinomialModel: deeplearning
Model Key:  DeepLearning_model_R_1527753908592_516 
Status of Neuron Layers: predicting volact, 2-class classification, bernoulli distribution, CrossEntropy loss, 70.702 weights/biases, 846,3 KB, 510.484 training samples, mini-batch size 1
  layer units      type dropout       l1       l2 mean_rate rate_rms momentum
1     1    50     Input  0.00 %                                              
2     2   200 Rectifier  0.00 % 0.000000 0.000000  0.020129 0.016055 0.000000
3     3   200 Rectifier  0.00 % 0.000000 0.000000  0.007546 0.003815 0.000000
4     4   100 Rectifier  0.00 % 0.000000 0.000000  0.028831 0.038248 0.000000
5     5     2   Softmax         0.000000 0.000000  0.006125 0.003235 0.000000
  mean_weight weight_rms mean_bias bias_rms
1                                          
2   -0.086518   0.350263  0.256193 0.160966
3   -0.029373   0.081107  0.909756 0.122269
4   -0.006860   0.087216 -0.259202 0.340486
5   -0.014144   0.370166 

In [49]:
nn1_1c_pred <- h2o.predict(nn1_1c, h2o.test1c)
nn6_1c_pred <- h2o.predict(nn6_1c, h2o.test6c)
ACC(table(as.vector(nn1_1c_pred$predict), test1c$volact))
PRE(table(as.vector(nn1_1c_pred$predict), test1c$volact))
F1(table(as.vector(nn1_1c_pred$predict), test1c$volact))
ACC(table(as.vector(nn6_1c_pred$predict), test6c$volact))
PRE(table(as.vector(nn6_1c_pred$predict), test6c$volact))
F1(table(as.vector(nn6_1c_pred$predict), test6c$volact))

  |======================================================================| 100%
  |======================================================================| 100%


[1] 0.6456563

[1] -0.07598566

[1] 0.5996266

[1] 0.625823

[1] -0.02402402

[1] 0.6132325

# random forest

In [16]:
rf1_1 <- h2o.randomForest(        
  training_frame = h2o.train1,        
  validation_frame = h2o.test1,      
  x = xnames1,                        
  y = "volact",                          
  model_id = "forest1",    
  ntrees = 300,            
  max_depth = 50,
  stopping_rounds = 2,           
  score_each_iteration = F,     
  seed = 1000000) 

rf6_1 <- h2o.randomForest(        
  training_frame = h2o.train6,        
  validation_frame = h2o.test6,      
  x = xnames6,                        
  y = "volact",                          
  model_id = "forest1",    
  ntrees = 300,            
  max_depth = 50,
  stopping_rounds = 2,           
  score_each_iteration = F,     
  seed = 1000000) 

  |======================================================================| 100%
  |======================================================================| 100%


In [17]:
summary(rf1_1)

Model Details:

H2OBinomialModel: drf
Model Key:  forest1 
Model Summary: 
  number_of_trees number_of_internal_trees model_size_in_bytes min_depth
1             300                      300            28644740        29
  max_depth mean_depth min_leaves max_leaves mean_leaves
1        42   33.87667       7438       7829  7604.64360

H2OBinomialMetrics: drf
** Reported on training data. **
** Metrics reported on Out-Of-Bag training samples **

MSE:  0.2024681
RMSE:  0.4499645
LogLoss:  0.5901142
Mean Per-Class Error:  0.376316
AUC:  0.6831484
Gini:  0.3662967

Confusion Matrix (vertical: actual; across: predicted) for F1-optimal threshold:
           0     1    Error          Rate
0      11517 13873 0.546396  =13873/25390
1       2626 10107 0.206236   =2626/12733
Totals 14143 23980 0.432783  =16499/38123

Maximum Metrics: Maximum metrics at their respective thresholds
                        metric threshold    value idx
1                       max f1  0.288550 0.550595 253
2          

In [18]:
summary(rf6_1)

Model Details:

H2OBinomialModel: drf
Model Key:  forest1 
Model Summary: 
  number_of_trees number_of_internal_trees model_size_in_bytes min_depth
1             300                      300            37452507        29
  max_depth mean_depth min_leaves max_leaves mean_leaves
1        42   33.50333       9766      10168  9944.08700

H2OBinomialMetrics: drf
** Reported on training data. **
** Metrics reported on Out-Of-Bag training samples **

MSE:  0.2014647
RMSE:  0.4488482
LogLoss:  0.5881973
Mean Per-Class Error:  0.3599372
AUC:  0.7075468
Gini:  0.4150935

Confusion Matrix (vertical: actual; across: predicted) for F1-optimal threshold:
           0     1    Error          Rate
0      15534 16171 0.510046  =16171/31705
1       3672 13828 0.209829   =3672/17500
Totals 19206 29999 0.403272  =19843/49205

Maximum Metrics: Maximum metrics at their respective thresholds
                        metric threshold    value idx
1                       max f1  0.304571 0.582244 253
2         

In [19]:
rf1_1_pred <- h2o.predict(rf1_1, h2o.test1)
rf6_1_pred <- h2o.predict(rf6_1, h2o.test6)
ACC(table(as.vector(rf1_1_pred$predict), test1$volact))
PRE(table(as.vector(rf1_1_pred$predict), test1$volact))
F1(table(as.vector(rf1_1_pred$predict), test1$volact))
ACC(table(as.vector(rf6_1_pred$predict), test6$volact))
PRE(table(as.vector(rf6_1_pred$predict), test6$volact))
F1(table(as.vector(rf6_1_pred$predict), test6$volact))

  |======================================================================| 100%
  |======================================================================| 100%


[1] 0.5365911

[1] -0.4071685

[1] 0.5245822

[1] 0.6111924

[1] -0.06406406

[1] 0.5933435

In [20]:
rf1_1c <- h2o.randomForest(        
  training_frame = h2o.train1c,        
  validation_frame = h2o.test1c,      
  x = xnames1c,                        
  y = "volact",                          
  model_id = "forest1",    
  ntrees = 300,            
  max_depth = 50,
  stopping_rounds = 2,           
  score_each_iteration = F,     
  seed = 1000000) 

rf6_1c <- h2o.randomForest(        
  training_frame = h2o.train6c,        
  validation_frame = h2o.test6c,      
  x = xnames6c,                        
  y = "volact",                          
  model_id = "forest1",    
  ntrees = 300,            
  max_depth = 50,
  stopping_rounds = 2,           
  score_each_iteration = F,     
  seed = 1000000) 

  |======================================================================| 100%
  |======================================================================| 100%


In [21]:
summary(rf1_1c)

Model Details:

H2OBinomialModel: drf
Model Key:  forest1 
Model Summary: 
  number_of_trees number_of_internal_trees model_size_in_bytes min_depth
1             191                      191            16712151        36
  max_depth mean_depth min_leaves max_leaves mean_leaves
1        49   40.98953       6553       7218  6948.49200

H2OBinomialMetrics: drf
** Reported on training data. **
** Metrics reported on Out-Of-Bag training samples **

MSE:  0.1870665
RMSE:  0.4325119
LogLoss:  0.5458372
Mean Per-Class Error:  0.3259692
AUC:  0.7432151
Gini:  0.4864303

Confusion Matrix (vertical: actual; across: predicted) for F1-optimal threshold:
           0     1    Error          Rate
0      13276 12114 0.477117  =12114/25390
1       2226 10507 0.174821   =2226/12733
Totals 15502 22621 0.376151  =14340/38123

Maximum Metrics: Maximum metrics at their respective thresholds
                        metric threshold    value idx
1                       max f1  0.301569 0.594388 246
2         

In [22]:
summary(rf6_1c)

Model Details:

H2OBinomialModel: drf
Model Key:  forest1 
Model Summary: 
  number_of_trees number_of_internal_trees model_size_in_bytes min_depth
1             202                      202            24655745        35
  max_depth mean_depth min_leaves max_leaves mean_leaves
1        48   39.65842       9411       9910  9681.74700

H2OBinomialMetrics: drf
** Reported on training data. **
** Metrics reported on Out-Of-Bag training samples **

MSE:  0.195747
RMSE:  0.442433
LogLoss:  0.5738983
Mean Per-Class Error:  0.3383817
AUC:  0.7289596
Gini:  0.4579193

Confusion Matrix (vertical: actual; across: predicted) for F1-optimal threshold:
           0     1    Error          Rate
0      16803 14902 0.470021  =14902/31705
1       3618 13882 0.206743   =3618/17500
Totals 20421 28784 0.376385  =18520/49205

Maximum Metrics: Maximum metrics at their respective thresholds
                        metric threshold    value idx
1                       max f1  0.311284 0.599862 246
2           

In [23]:
rf1_1c_pred <- h2o.predict(rf1_1c, h2o.test1c)
rf6_1c_pred <- h2o.predict(rf6_1c, h2o.test6c)
ACC(table(as.vector(rf1_1c_pred$predict), test1c$volact))
PRE(table(as.vector(rf1_1c_pred$predict), test1c$volact))
F1(table(as.vector(rf1_1c_pred$predict), test1c$volact))
ACC(table(as.vector(rf6_1c_pred$predict), test6c$volact))
PRE(table(as.vector(rf6_1c_pred$predict), test6c$volact))
F1(table(as.vector(rf6_1c_pred$predict), test6c$volact))

  |======================================================================| 100%


Warning message in doTryCatch(return(expr), name, parentenv, handler):
"Test/Validation dataset is missing column 'cntryAL': substituting in a column of NaN"Warning message in doTryCatch(return(expr), name, parentenv, handler):
"Test/Validation dataset is missing column 'cntryBG': substituting in a column of NaN"Warning message in doTryCatch(return(expr), name, parentenv, handler):
"Test/Validation dataset is missing column 'cntryCY': substituting in a column of NaN"Warning message in doTryCatch(return(expr), name, parentenv, handler):
"Test/Validation dataset is missing column 'cntryEE': substituting in a column of NaN"Warning message in doTryCatch(return(expr), name, parentenv, handler):
"Test/Validation dataset is missing column 'cntryIS': substituting in a column of NaN"Warning message in doTryCatch(return(expr), name, parentenv, handler):
"Test/Validation dataset is missing column 'cntryLT': substituting in a column of NaN"Warning message in doTryCatch(return(expr), name, parenten

  |======================================================================| 100%


[1] 0.5573654

[1] -0.344086

[1] 0.533466

[1] 0.6486832

[1] 0.03853854

[1] 0.6190759